In [1]:
%matplotlib inline
import random
import math
import numpy as np
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit import *
from qiskit.visualization import *
# Loading your IBM Q account(s)
provider = IBMQ.load_account()

import numpy as np

from qiskit.providers.aer.noise import NoiseModel
from qiskit.aqua import QuantumInstance
from qiskit.providers.aer.noise import thermal_relaxation_error, ReadoutError, depolarizing_error

provider = IBMQ.get_provider(hub='ibm-q')
machine = provider.get_backend('ibmq_vigo')
# from qiskit.test.mock import FakeLondon
# machine = FakeLondon()
machine_noise_model = NoiseModel.from_backend(machine)
simulator = Aer.get_backend('qasm_simulator')


basis_gates = machine_noise_model.basis_gates

C:\Users\Eesh Gupta\anacondaaa\lib\site-packages\qiskit\providers\ibmq\ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


In [2]:
provider = IBMQ.get_provider(hub='ibm-q-education', group = 'rutgers-uni-Schn', project = 'vqe-research')
machine = provider.get_backend('ibmq_rome')
config = machine.configuration()
print("This backend is called {0}, and is on version {1}. It has {2} qubit{3}. It "
      "{4} OpenPulse programs. The basis gates supported on this device are {5}."
      "".format(config.backend_name,
                config.backend_version,
                config.n_qubits,
                '' if config.n_qubits == 1 else 's',
                'supports' if config.open_pulse else 'does not support',
                config.basis_gates))

This backend is called ibmq_rome, and is on version 1.1.4. It has 5 qubits. It does not support OpenPulse programs. The basis gates supported on this device are ['id', 'u1', 'u2', 'u3', 'cx'].


### Gate Insertion Functions

In [ ]:
from GateInsertion_funcs import splitter, PiReader, GateInsertion, u3PairAdder, cXPairAdder, GateInserter


### Pulse Stretching Functions

In [ ]:
from PulseStretching_funcs import Scheduler, stretchSchedule_singleQ

### Result Functions

In [ ]:
from Result_funcs import countsToProb, averagingSeeds, addDicts, runExperiments, getProb

## Randomized Benchmarking

In [ ]:
#Import general libraries (needed for functions)
import numpy as np
import matplotlib.pyplot as plt
from IPython import display

#Import the RB Functions
import qiskit.ignis.verification.randomized_benchmarking as rb

#Import Qiskit classes 
import qiskit
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors.standard_errors import depolarizing_error, thermal_relaxation_error

In [ ]:
#Generate RB circuits (2Q RB)

#number of qubits
nQ=1
rb_opts = {}
#Number of Cliffords in the sequence
rb_opts['length_vector'] = [1] +[10*i for i in range(1, 16)]
#Number of seeds (random sequences)
rb_opts['nseeds'] =1
#Default pattern
rb_opts['rb_pattern'] = [[0]]

rb_circs, xdata = rb.randomized_benchmarking_seq(**rb_opts)

In [ ]:
# converting everything to u3

from qiskit.compiler import transpile
basis_gates =  ['id', 'u3']

rb_u3_circs = []

for i in range(len(rb_circs)): 
    seed_circs = []
    for j in range(len(rb_circs[i])): 
        transpiled_circ = transpile(rb_circs[i][j], basis_gates = basis_gates)
        seed_circs.append(transpiled_circ)
    rb_u3_circs.append(seed_circs)


In [ ]:
#Test
rb_u3_circs[0][1].draw(output = 'mpl')

In [ ]:
circ = GateInsertion(rb_u3_circs[0][1], 1.1)
circ.draw(output = 'mpl')

### Specify Scale Factors

In [ ]:
scales= [1, 1.1, 1.2, 1.3, 1.4]

In [ ]:
scaled_circuits_Gate = GateInserter(scales, rb_u3_circs)
scaled_circuits_Pulse = Scheduler(scales, rb_u3_circs, machine)

### Results

In [ ]:
from Result_funcs import countsToProb, averagingSeeds, addDicts, runExperiments

In [ ]:
#running the jobs 
Probs = [] # buckets are scales, each of which contain probs for various gate lengths 

from qiskit.tools.monitor import job_monitor
Probs = []
samples = 10

#running all experiments 
counts = runExperiments(scaled_circuits_Gate, simulator,samples)
#print(counts)



In [ ]:
Probs = getProb(counts, samples)

## Plotting Results

In [ ]:
import matplotlib.pyplot as plt 

a = plt.figure()

for scale in range(len(Probs)): 
    plt.plot(rb_opts['length_vector'], Probs[scale], marker = 'o', linestyle= '--', label = str(scales[scale]))
plt.title('Gate Insertion')
plt.legend()
plt.show()
#a.savefig("plotSep30(1).png", dpi = 1000)

In [ ]:
Probs

In [ ]:
for i in range(length_vector):
    gate_data = []
    for j in range(scales):
        gate_data.append(Probs[j][i])
    
        